In [1]:
try:
  import mlagents
  print("ml-agents already installed")
except ImportError:
  !python -m pip install -q mlagents==0.28.0
  print("Installed ml-agents")

ml-agents already installed


In [2]:
#@title Select Environment { display-mode: "form" }
env_id = "TouchCube"

In [14]:
try:
    env.close()
except:
    pass

from mlagents_envs.environment import UnityEnvironment
print(UnityEnvironment.API_VERSION)


1.5.0


In [ ]:
env = UnityEnvironment(file_name="TouchCube", worker_id=1)

In [7]:
env.reset()

In [8]:
# We will only consider the first Behavior
behavior_name = list(env.behavior_specs)[0]
print(f"Name of the behavior : {behavior_name}")
spec = env.behavior_specs[behavior_name]

Name of the behavior : TouchCube?team=0


In [9]:
# Examine the number of observations per Agent
print("Number of observations : ", len(spec.observation_specs))

# Is there a visual observation ?
# Visual observation have 3 dimensions: Height, Width and number of channels
vis_obs = any(len(spec.shape) == 3 for spec in spec.observation_specs)
print("Is there a visual observation ?", vis_obs)

Number of observations :  2
Is there a visual observation ? False


In [10]:
# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
  print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
  print(f"There are {spec.action_spec.discrete_size} discrete actions")


# How many actions are possible ?
#print(f"There are {spec.action_size} action(s)")

# For discrete actions only : How many different options does each action has ?
if spec.action_spec.discrete_size > 0:
  for action, branch_size in enumerate(spec.action_spec.discrete_branches):
    print(f"Action number {action} has {branch_size} different options")

There are 7 discrete actions
Action number 0 has 3 different options
Action number 1 has 3 different options
Action number 2 has 3 different options
Action number 3 has 3 different options
Action number 4 has 3 different options
Action number 5 has 3 different options
Action number 6 has 3 different options


In [11]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [12]:
for episode in range(3):
  env.reset()
  decision_steps, terminal_steps = env.get_steps(behavior_name)
  tracked_agent = -1 # -1 indicates not yet tracking
  done = False # For the tracked_agent
  episode_rewards = 0 # For the tracked_agent
  while not done:
    # Track the first agent we see if not tracking
    # Note : len(decision_steps) = [number of agents that requested a decision]
    if tracked_agent == -1 and len(decision_steps) >= 1:
      tracked_agent = decision_steps.agent_id[0]

    # Generate an action for all agents
    action = spec.action_spec.random_action(len(decision_steps))

    # Set the actions
    env.set_actions(behavior_name, action)

    # Move the simulation forward
    env.step()

    # Get the new simulation results
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    if tracked_agent in decision_steps: # The agent requested a decision
      episode_rewards += decision_steps[tracked_agent].reward
    if tracked_agent in terminal_steps: # The agent terminated its episode
      episode_rewards += terminal_steps[tracked_agent].reward
      done = True
  print(f"Total rewards for episode {episode} is {episode_rewards}")

Total rewards for episode 0 is -3.1571199391037226
Total rewards for episode 1 is -10.58250948227942
Total rewards for episode 2 is -15.445563223212957


In [13]:
env.close()
print("Closed environment")

Closed environment
